In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -u ./drive/MyDrive/facial-keypoints-detection/test.zip
!unzip -u ./drive/MyDrive/facial-keypoints-detection/training.zip
!cp ./drive/MyDrive/facial-keypoints-detection/IdLookupTable.csv ./

Archive:  ./drive/MyDrive/facial-keypoints-detection/test.zip
Archive:  ./drive/MyDrive/facial-keypoints-detection/training.zip


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.layers.advanced_activations import LeakyReLU
from keras.layers import ReLU, UpSampling2D, Add, Input, Activation, Convolution2D, LSTM, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D, MaxPool2D, ZeroPadding2D
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam, RMSprop
import tensorflow as tf
from tensorflow.math import pow, log, abs, cumsum, reduce_sum, greater, reduce_mean


In [4]:
remove_nulls = True

NUM_EPOCHS = 5
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.1

stacked_hourglass_network = True
sequential_network = False

visualize_data = False

In [5]:
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('test.csv')
idlookup_data = pd.read_csv('IdLookupTable.csv')
checkpoint_filepath = '/tmp/checkpoint'

In [6]:
# 13 facial keypoints and then the image
train_data.head(5).shape

(5, 31)

In [7]:
if remove_nulls: 
  train_data = train_data.dropna()

In [8]:
def adaptive_wing_loss(labels, output):
    alpha = 2.1
    omega = 14
    epsilon = 1
    theta = 0.5
    with tf.name_scope('adaptive_wing_loss'):
        x = output - labels
        theta_over_epsilon_tensor = tf.fill(tf.shape(labels), theta/epsilon)
        A = omega*(1/(1+pow(theta_over_epsilon_tensor, alpha-labels)))*(alpha-labels)*pow(theta_over_epsilon_tensor, alpha-labels-1)*(1/epsilon)
        C = theta*A-omega*log(1+pow(theta_over_epsilon_tensor, alpha-labels))
        absolute_x = abs(x)
        losses = tf.where(greater(theta, absolute_x), omega*log(1+pow(absolute_x/epsilon, alpha-labels)), A*absolute_x-C)
        loss = reduce_mean(reduce_sum(losses, axis=[1, 2]), axis=0)
        return loss

In [9]:
# def fetch_images(input_data, image_dims):
#   output = []
#   for index, row in input_data.iterrows():
#     raw_image = row['Image'].split(' ')
#     processed_image = np.reshape(np.array(raw_image, dtype=int), image_dims)
#     output.append(processed_image)
#   return np.array(output)/255.

# def fetch_keypoints(input_data):
#   output = []
#   # last column is image data
#   raw_keypoints = input_data.iloc[: , :-1]
#   return raw_keypoints.to_numpy(dtype = 'float')

def fetch_images(image_data, image_dims):
    images = []
    for idx, sample in image_data.iterrows():
        image = np.array(sample['Image'].split(' '), dtype=int)
        image = np.reshape(image, (96,96,1))
        img_pad = np.zeros((128, 128, 1))
        img_pad[:image.shape[0],:image.shape[1]] = image
        images.append(img_pad)
    images = np.array(images)/255.
    return images

def fetch_keypoints(keypoint_data):
    keypoint_data = keypoint_data.drop('Image',axis = 1)
    keypoint_features = []
    for idx, sample_keypoints in keypoint_data.iterrows():
        keypoint_features.append(sample_keypoints)
    keypoint_features = np.array(keypoint_features, dtype = 'float')
    return keypoint_features

In [10]:
model_checkpoint_callback = ModelCheckpoint(filepath = 'checkpoint_filepath', 
                                            monitor='val_mse', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='min')


In [11]:
def sequential_nn():
  model = Sequential()

  # Input dimensions: (None, 96, 96, 1)
  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(128,128,1)))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 96, 96, 32)
  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Input dimensions: (None, 48, 48, 32)
  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 48, 48, 64)
  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Input dimensions: (None, 24, 24, 64)
  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 24, 24, 96)
  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Input dimensions: (None, 12, 12, 96)
  model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 12, 12, 128)
  model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Input dimensions: (None, 6, 6, 128)
  model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 6, 6, 256)
  model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Input dimensions: (None, 3, 3, 256)
  model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  # Input dimensions: (None, 3, 3, 512)
  model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  # Input dimensions: (None, 3, 3, 512)
  model.add(Flatten())
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(30))
  
  return model


In [49]:
def bottleneck(inputs, filters, downsample=False):
  identity = inputs

  if downsample:
      identity = Conv2D(
          filters=filters,  # lift channels first
          kernel_size=1,
          padding='same')(inputs)

  x = BatchNormalization(momentum=0.9)(inputs)
  x = ReLU()(x)
  # kernel_initializer='he_normal'
  x = Conv2D(filters=filters // 2, kernel_size=1, padding='same')(x)

  x = BatchNormalization(momentum=0.9)(x)
  x = ReLU()(x)
  # kernel_initializer='he_normal'
  x = Conv2D(filters=filters // 2, kernel_size=3, padding='same')(x)

  x = BatchNormalization(momentum=0.9)(x)
  x = ReLU()(x)
  # kernel_initializer='he_normal'
  x = Conv2D(filters=filters, kernel_size=1, padding='same')(x)
  
  x = Add()([identity, x])
  return x

def hourglass(inputs, order, filters, num_residual):
  # Upper branch maintaining the current image
  up1 = bottleneck(inputs, filters)
  # print("Initial Up1")
  # print(up1.shape)

  for i in range(num_residual):
    # print("Up1 #", str(i))
    # print(up1.shape)
    up1 = bottleneck(up1, filters)
  # print("Final up")
  # print(up1.shape)
  # Lower branch with actual hourglasses
  
  low1 = MaxPool2D(pool_size=2, strides=2)(inputs)
  for i in range(num_residual):
      low1 = bottleneck(low1, filters, downsample=False)

  low2 = low1
  if order > 1:
      low2 = hourglass(low1, order - 1, filters, num_residual)
  else:
      for i in range(num_residual):
          low2 = bottleneck(low2, filters, downsample=False)

  low3 = low2
  for i in range(num_residual):
      low3 = bottleneck(low3, filters, downsample=False)

  up2 = UpSampling2D(size=2)(low3)

  # print("Up sizes")
  # print(up1.shape)
  # print("Up2")
  # print(up2.shape)

  # print("low sizes")
  # print(low1.shape)
  # print(low2.shape)
  # print(low3.shape)

  return up2 + up1


def linear_layer(inputs, filters):
    x = Conv2D(
        filters=filters,
        kernel_size=1,
        padding='same')(inputs)
    x = BatchNormalization(momentum=0.9)(x)
    x = ReLU()(x)
    return x

def stacked_hourglass(input_shape=(128, 128, 1), num_stack=2, num_residual=1, num_heatmap=128):
  inputs = Input(shape=input_shape)

  # initial processing of the image
  x = Conv2D(
      filters=32,
      kernel_size=7,
      strides=2,
      padding='same',
      kernel_initializer='he_normal')(inputs)
  x = BatchNormalization(momentum=0.9)(x)
  x = ReLU()(x)
  x = bottleneck(x, 64, downsample=True)
  x = MaxPool2D(pool_size=2, strides=2)(x)
  x = bottleneck(x, 64, downsample=False)
  x = bottleneck(x, 128, downsample=True)


  ys = []
  for i in range(num_stack):
      x = hourglass(x, order=4, filters=128, num_residual=num_residual)
      for i in range(num_residual):
          x = bottleneck(inputs=x, filters=128)

      x = linear_layer(x, 128)

      y = Conv2D(filters=30, kernel_size=3, padding='same')(x)
      ys.append(y)


      # if it's not the last stack, we need to add predictions back
      if i < num_stack - 1:
        y_intermediate_1 = Conv2D(filters=512, kernel_size=1)(x)
        y_intermediate_2 = Conv2D(filters=512, kernel_size=1)(y)
        # print("inbetween sizes")
        # print(y_intermediate_1.shape)
        # print(y_intermediate_2.shape)
        x = Add()([y_intermediate_1, y_intermediate_2])

  z = Flatten()(ys[-1])
  z = Dense(512,activation='relu')(z)
  z = Dropout(0.1)(z)
  z = Dense(30)(z)
  ys.append(z)
  # print(ys)
  return Model(inputs, ys, name='stacked_hourglass')


Sequential Model Layers: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 128, 128, 32)      288       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9216      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
.......

conv2d_11 (Conv2D)           (None, 4, 4, 512)         2359296   
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 4, 4, 512)         0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 4, 4, 512)         2048      
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                15390 

Model: "stacked_hourglass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_4 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_216 (Conv2D)             (None, 64, 64, 32)   1600        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_207 (BatchN (None, 64, 64, 32)   128         conv2d_216[0][0]                 
__________________________________________________________________________________________________
re_lu_195 (ReLU)                (None, 64, 64, 32)   0           batch_normalization_207[0][0]    
__________________________________________________________________________________________________
batch_normalization_208 (BatchN (None, 64, 64, 32)   128         re_lu_195[0][0]                  
__________________________________________________________________________________________________
......

conv2d_287 (Conv2D)             (None, 16, 16, 512)  2359296     batch_normalization_274[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_15 (LeakyReLU)      (None, 16, 16, 512)  0           conv2d_287[0][0]                 
__________________________________________________________________________________________________
batch_normalization_275 (BatchN (None, 16, 16, 512)  2048        leaky_re_lu_15[0][0]             
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 30720)        0           conv2d_355[0][0]                 
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 512)          15729152    flatten_4[0][0]                  
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 512)          0           dense_9[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 30)           15390       dropout_4[0][0]              
__________________________________________________________________________________________________
Total params: 72,468,224
Trainable params: 72,454,784
Non-trainable params: 13,440

In [39]:
my_optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#my_optimizer = RMSprop(2.5e-4)

In [50]:
if stacked_hourglass_network:
  model = stacked_hourglass(input_shape=(128, 128, 1), num_stack=1, num_residual=1)

elif sequential_network:
  model = sequential_nn()

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse', 'acc', RootMeanSquaredError()])

In [41]:
# training_input = fetch_images(train_data, (128, 128, 1))
# training_output = fetch_keypoints(train_data)

In [51]:
print(model.summary())
print(type(training_input))
print(training_input.shape)
print(type(training_output))
print(training_output.shape)


Model: "stacked_hourglass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_288 (Conv2D)             (None, 64, 64, 32)   1600        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_276 (BatchN (None, 64, 64, 32)   128         conv2d_288[0][0]                 
__________________________________________________________________________________________________
re_lu_260 (ReLU)                (None, 64, 64, 32)   0           batch_normalization_276[0][0]    
__________________________________________________________________________________

In [43]:
def plot_sample(image, keypoint, axis, title):
    image = image.reshape(128,128)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], marker='x', s=20)
    plt.title(title)

In [44]:
#Visualize data
if visualize_data:
  fig = plt.figure(figsize=(20,8))
  for i in range(10):
      axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
      plot_sample(training_input[i], training_output[i], axis, "")
  plt.show()


In [45]:
history = model.fit(training_input, training_output, epochs=NUM_EPOCHS, 
                    batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT, 
                    verbose=1, callbacks=[model_checkpoint_callback])

Epoch 1/5


ValueError: ignored